### HW 3.0
__What is a merge sort? Where is it used in Hadoop?__



The merge sort is a common sorting algorithm. In this design, one defines three references at the front of each array. So one reference at the front of each array with existing values and a reference at the front of the new fully sorted array. Then the algorithm picks the smallest element from the two existing lists and adds it to the temporary array, moving the appropriate references. 
Hadoop uses merge sort in the shuffling phase. The shuffle phase is divided in three parts: partition, sort, combine. At the shuffle, records are sorted within each partition (more on this in a later question) and merge sort is used to combine the sorted records. This occurs after the mapper has produced its output but before the output gets to the reducer

__How is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.__

A combiner function is essentially a function that runs in between the mapper and reducer to reduce writing to disk and reduce network traffic. A good example is in a word count with multiple mappers and reducers. Each mapper outputs some tokens and associated counts as key-value pairs. In the reducer, these tokens will be used as keys and the counts added up to produce totals. In this context, a combiner would be a second version of the reducer running before the real reducer. What this will do is aggregate the counts for tokens that exit the mapper but before they are fully output at the reducer. The point here is it means less of the partition is saved to disk by reducing duplication from the mapper as well as there is less communication between the mapper and reducer. 

__What is the Hadoop shuffle?__

The Hadoop shuffle is all the steps taken between the mapper output and the reducer input in a MapReduce job. This includes, partitioning output, sorting the output, combining output (all from the mapper), merging the sorted outputs, sending this to the reducer, mergesort on the partition files, and streaming to the reducer. 

### HW3.1 Use Counters to do EDA (exploratory data analysis and to monitor progress)

Counters are lightweight objects in Hadoop that allow you to keep track of system progress in both the map and reduce stages of processing. By default, Hadoop defines a number of standard counters in "groups"; these show up in the jobtracker webapp, giving you information such as "Map input records", "Map output records", etc. 

While processing information/data using MapReduce job, it is a challenge to monitor the progress of parallel threads running across nodes of distributed clusters. Moreover, it is also complicated to distinguish between the data that has been processed and the data which is yet to be processed. The MapReduce Framework offers a provision of user-defined Counters, which can be effectively utilized to monitor the progress of data across nodes of distributed clusters.

Use the Consumer Complaints  Dataset provide here to complete this question:

     https://www.dropbox.com/s/vbalm3yva2rr86m/Consumer_Complaints.csv?dl=0

The consumer complaints dataset consists of diverse consumer complaints, which have been reported across the United States regarding various types of loans. The dataset consists of records of the form:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

Here’s is the first few lines of the  of the Consumer Complaints  Dataset:

Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,Company,Company response,Timely response?,Consumer disputed?

1114245,Debt collection,Medical,Disclosure verification of debt,Not given enough info to verify debt,FL,32219,Web,11/13/2014,11/13/2014,"Choice Recovery, Inc.",Closed with explanation,Yes,

1114488,Debt collection,Medical,Disclosure verification of debt,Right to dispute notice not received,TX,75006,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

1114255,Bank account or service,Checking account,Deposits and withdrawals,,NY,11102,Web,11/13/2014,11/13/2014,"FNIS (Fidelity National Information Services, Inc.)",In progress,Yes,

1115106,Debt collection,"Other (phone, health club, etc.)",Communication tactics,Frequent or repeated calls,GA,31721,Web,11/13/2014,11/13/2014,"Expert Global Solutions, Inc.",In progress,Yes,

User-defined Counters

Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 


In [1]:
%%writefile mapper.py
#!/usr/bin/python
import sys

line_num = 0
for line in sys.stdin: 
    if line_num == 0:
        line_num += 1
        continue
    else:
        line = line.strip()
        line = line.rstrip()
        line = line.split(',')
        if line[1] == "Debt collection":
            sys.stderr.write('reporter:counter:Debt-Counter,Total,1\n')
        elif line[1] == 'Mortgage':
            sys.stderr.write('reporter:counter:Mortgage-Counter,Total,1\n')
        else:
            sys.stderr.write('reporter:counter:Other-Counter,Total,1\n')
        sys.stderr.write("reporter:counter:Tokens,Total,1\n")
        print line[1] + '\t' + '1'

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/python
import sys

debt_counter = 0
mortgage_counter = 0
other_counter = 0

for line in sys.stdin:
    line = line.strip().split('\t')
    if line[0] == "Debt collection":
        debt_counter +=1 
    elif line[0] == 'Mortgage':
        mortgage_counter += 1
    else:
        other_counter += 1
print "Debt collection: " + str(debt_counter)
print "Mortgage: " + str(mortgage_counter)
print "Other: " + str(other_counter)

Overwriting reducer.py


In [8]:
#!cat Consumer_Complaints.csv | python mapper.py | python reducer.py

In [3]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/01/29 21:04:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/01/29 21:05:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [4]:
#add input to hdfs
!hdfs dfs -put Consumer_Complaints.csv /user/dunmireg

16/01/29 21:05:29 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper.py \
-reducer reducer.py \
-input Consumer_Complaints.csv \
-output consumer_counters

16/01/29 21:05:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/29 21:05:34 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/29 21:05:34 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/29 21:05:34 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/29 21:05:35 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/29 21:05:35 INFO mapreduce.JobSubmitter: number of splits:1
16/01/29 21:05:35 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1206054309_0001
16/01/29 21:05:35 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/29 21:05:35 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/29 21:05:35 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.Fi

In [6]:
#show results
!hdfs dfs -cat /user/dunmireg/consumer_counters/part-00000

16/01/29 21:06:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Debt collection: 44372	
Mortgage: 125752	
Other: 142789	


In [7]:
#Start history server to check counter
!/usr/local/Cellar/hadoop/2.7.1/sbin/mr-jobhistory-daemon.sh --config /usr/local/Cellar/hadoop/2.7.1/libexec/etc/hadoop start historyserver

starting historyserver, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/mapred-dunmireg-historyserver-Glenns-Air.home.out


In [8]:
#Stop history server
!/usr/local/Cellar/hadoop/2.7.1/sbin/mr-jobhistory-daemon.sh --config /usr/local/Cellar/hadoop/2.7.1/libexec/etc/hadoop stop historyserver

stopping historyserver


Use this link to view job tracker

http://192.168.0.10:19888/jobhistory/app

In [22]:
#move output to local directory
#!hadoop fs -copyToLocal /user/dunmireg/consumer_counters

16/01/29 18:22:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/29 18:22:50 WARN hdfs.DFSClient: DFSInputStream has been closed already
16/01/29 18:22:50 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [9]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/Consumer_Complaints.csv
!hadoop fs -rmr /user/dunmireg/consumer_counters
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/01/29 21:09:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/29 21:09:45 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/Consumer_Complaints.csv
rmr: DEPRECATED: Please use 'rm -r' instead.
16/01/29 21:09:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/29 21:09:47 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/consumer_counters
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
localhost: nodemanager did not stop gracefully after 5 seconds: killing with kill -9
no proxyserver to stop
16/01/29 21:09:58 WARN util.NativeCodeLoader: Unable to load nati

### HW 3.2 Analyze the performance of your Mappers, Combiners and Reducers using Counters

__Part 1__

For this brief study the Input file will be one record (the next line only): 
foo foo quux labs foo bar quux


Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer  should be 1 and 4 respectively. Please explain.


In [1]:
with open('input_text.txt', 'w') as myfile:
    myfile.write('foo foo quux labs foo bar quux')

In [8]:
%%writefile mapper.py
#!/usr/bin/python
import sys

sys.stderr.write('reporter:counter:Map-Count,Total,1\n')
for line in sys.stdin:
    line = line.strip()
    line = line.rstrip()
    line = line.split()
    for word in line:
        print word + '\t' + '1'

Overwriting mapper.py


In [9]:
%%writefile reducer.py
#!/usr/bin/python
import sys

current_word = None
current_count = None
word = None
sys.stderr.write('reporter:counter:Reduce-Counter,Total,1\n')
for line in sys.stdin:
    line = line.strip()
    line = line.rstrip()
    word, count = line.split('\t')
    
    if current_word == word:
        current_count += int(count)
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_word = word
        current_count = int(count)

if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [19]:
#!cat input.txt | python mapper.py | sort | python reducer.py

In [4]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 22:31:34 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 22:31:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [5]:
#add input to hdfs
!hdfs dfs -put input_text.txt /user/dunmireg

16/02/01 22:31:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.map.tasks=1 \
-D mapred.reduce.tasks=4 \
-mapper mapper.py \
-reducer reducer.py \
-input input_text.txt \
-output word_count

16/02/01 22:33:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:33:49 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:33:49 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:33:49 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:33:49 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:33:49 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:33:49 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/02/01 22:33:49 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 22:33:49 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local927319060_0001
16/02/01 22:33:49 INFO mapr

In [12]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/input_text.txt
!hadoop fs -rmr /user/dunmireg/word_count
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:34:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:34:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/input_text.txt
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:34:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:34:10 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/word_count
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/02/01 22:34:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes

__Part 2__

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).
Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [15]:
%%writefile mapper.py
#!/usr/bin/python
import sys
import re
from csv import reader

#Structure of complaints
#Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,
#Company,Company response,Timely response?,Consumer disputed?
line_num = 0

WORD_RE = re.compile(r"[\w']+")
sys.stderr.write('reporter:counter:Map-Count,Total,1\n')
for line in reader(sys.stdin):
    if line_num == 0:
        line_num += 1
        continue
    else:
        issue = line[3]
        if issue == '':
            issue = 'Blank'
        words = re.findall(WORD_RE, issue)
        for word in words:
            print word.lower() + '\t' + str(1)

Overwriting mapper.py


In [16]:
%%writefile reducer.py
#!/usr/bin/python
import sys

current_word = None
current_count = None
word = None
sys.stderr.write('reporter:counter:Reduce-Counter,Total,1\n')
for line in sys.stdin:
    #issue, count = line.split('\t')
    line = line.split('\t')
    word = line[0]
    count = int(line[1])
    
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_word = word
        current_count = count

if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [6]:
#!cat Consumer_Complaints.csv | python mapper.py | sort | python reducer.py

In [3]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 17:40:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 17:40:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [4]:
#add input to hdfs
!hdfs dfs -put Consumer_Complaints.csv /user/dunmireg

16/02/01 17:40:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.reduce.tasks=2 \
-D mapred.map.tasks=2 \
-mapper mapper.py \
-reducer reducer.py \
-input Consumer_Complaints.csv \
-output word_count

16/02/01 17:40:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:40:31 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:40:31 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:40:31 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:40:31 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:40:31 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:40:32 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/02/01 17:40:32 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 17:40:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local161210426_0001
16/02/01 17:40:32 INFO mapr

In [7]:
#show results
#!hdfs dfs -cat /user/dunmireg/word_count/part-00000

In [6]:
#move output to local directory
!hadoop fs -copyToLocal /user/dunmireg/word_count

16/02/01 17:41:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:41:11 WARN hdfs.DFSClient: DFSInputStream has been closed already
16/02/01 17:41:11 WARN hdfs.DFSClient: DFSInputStream has been closed already
16/02/01 17:41:11 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [7]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/Consumer_Complaints.csv
!hadoop fs -rmr /user/dunmireg/word_count
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:41:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:41:14 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/Consumer_Complaints.csv
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:41:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:41:16 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/word_count
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/02/01 17:41:28 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping 

__Part 3__

Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job. 

In [13]:
%%writefile combiner.py
#!/usr/bin/python
import sys

current_word = None
current_count = None
word = None
sys.stderr.write('reporter:counter:Combiner-Counter,Total,1\n')
for line in sys.stdin:
    #issue, count = line.split('\t')
    line = line.split('\t')
    word = line[0]
    count = int(line[1])
    
    if current_word == word:
        current_count += count
    else:
        if current_word:
            print '%s\t%s' % (current_word, current_count)
        current_word = word
        current_count = count

if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Writing combiner.py


In [14]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 22:35:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 22:36:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [17]:
#add input to hdfs
!hdfs dfs -put Consumer_Complaints.csv /user/dunmireg

16/02/01 22:36:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.reduce.tasks=2 \
-D mapred.map.tasks=2 \
-mapper mapper.py \
-reducer reducer.py \
-combiner combiner.py \
-input Consumer_Complaints.csv \
-output word_count

16/02/01 22:36:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:36:51 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:36:51 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:36:51 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:36:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:36:51 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:36:52 INFO Configuration.deprecation: mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
16/02/01 22:36:52 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
16/02/01 22:36:52 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1588555268_0001
16/02/01 22:36:52 INFO map

In [5]:
#show results
#!hdfs dfs -cat /user/dunmireg/word_count/part-00000

In [19]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/Consumer_Complaints.csv
!hadoop fs -rmr /user/dunmireg/word_count
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:37:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:37:12 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/Consumer_Complaints.csv
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:37:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:37:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/word_count
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/02/01 22:37:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping 

__Part 4__

Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). 

In [2]:
%%writefile mapper-3-2-4.py
#!/usr/bin/python
import sys
import re
from csv import reader

#Structure of complaints
#Complaint ID,Product,Sub-product,Issue,Sub-issue,State,ZIP code,Submitted via,Date received,Date sent to company,
#Company,Company response,Timely response?,Consumer disputed?
line_num = 0
total = 0
WORD_RE = re.compile(r"[\w']+")
for line in reader(sys.stdin):
    if line_num == 0:
        line_num += 1
        continue
    else:
        issue = line[3]
        if issue == '':
            issue = 'Blank'
        words = re.findall(WORD_RE, issue)
        for word in words:
            total += 1
            print word.lower() + '\t' + str(1)
print '*' + '\t' + str(total)

Overwriting mapper-3-2-4.py


In [7]:
%%writefile reducer-3-2-4.py
#!/usr/bin/python
import sys
import operator

current_word = None
current_count = None
word = None
total = 0
wordcount = {}

for line in sys.stdin:
    line = line.split('\t')
    word = line[0]
    count = int(line[1])
    if word == '*':
        total = count
    else:
        if current_word == word:
            current_count += count
        else:
            if current_word:
                #sys.stderr.write('reporter:counter:Reduce-Counter,Total,1\n')
                #wordcount[current_word] = current_count
                print current_word + '\t' + str(current_count) + '\t' + str(float(current_count)/total)
            current_word = word
            current_count = count

if current_word == word:
    #sys.stderr.write('reporter:counter:Reduce-Counter,Total,1\n')
    #wordcount[current_word] = current_count
    print current_word + '\t' + str(current_count) + '\t' + str(float(current_count)/total)
    
# largest = 50
# smallest = 10
# sortedWordCount = sorted(wordcount.items(), key = operator.itemgetter(1))

# print "The Top 50 terms are"
# for i in range(largest):
#     print str(sortedWordCount[-i-1][0]) + '\t' + str(sortedWordCount[-i-1][1]) + '\t' + str(float(sortedWordCount[-i-1][1])/total)
    
# print '\n'

# print "The bottom 10 terms are"
# for i in range(smallest):
#     print str(sortedWordCount[i][0]) + '\t' + str(sortedWordCount[i][1]) + '\t' + str(float(sortedWordCount[i][1])/total)

Overwriting reducer-3-2-4.py


In [3]:
%%writefile mapper-3-2-4-2.py
#!/usr/bin/python
import sys

#Structure is word + \t + count + \t + relative count

for line in sys.stdin:
    line = line.strip()
    line = line.split('\t')
    print line[1] + '\t'+ line[0] + '\t' + line[2]


Overwriting mapper-3-2-4-2.py


In [10]:
%%writefile reducer-3-2-4-2.py
#!/usr/bin/python
import sys

print "Issue" + '\t' + "Count" + '\t' + 'Relative Count'
for line in sys.stdin:
    line = line.strip()
    line = line.split('\t')
    #structure currently is count + word + relative count
    print line[1] + '\t' + line[0] + '\t'+ line[2]

Overwriting reducer-3-2-4-2.py


In [11]:
#!cat Consumer_Complaints.csv | python mapper-3-2-4.py | sort | python reducer-3-2-4.py | python mapper-3-2-4-2.py | sort -n | python reducer-3-2-4-2.py> output.txt

In [12]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/01/31 23:00:49 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/01/31 23:01:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [13]:
#add input to hdfs
!hdfs dfs -put Consumer_Complaints.csv /user/dunmireg

16/01/31 23:01:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-2-4.py \
-reducer reducer-3-2-4.py \
-input Consumer_Complaints.csv \
-output word_count

16/01/31 23:01:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:01:12 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 23:01:12 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 23:01:12 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 23:01:13 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 23:01:13 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 23:01:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1964160718_0001
16/01/31 23:01:13 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/01/31 23:01:13 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/01/31 23:01:13 INFO mapreduce.Job: Running job: job_local1964160718_0001
16/01/31 23:01:1

In [15]:
#run second job to properly sort by counts
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D  mapred.text.key.comparator.options=-n \
-mapper mapper-3-2-4-2.py \
-reducer reducer-3-2-4-2.py \
-input /user/dunmireg/word_count/part-00000 \
-output sortedWordCount

16/01/31 23:01:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:01:45 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/01/31 23:01:45 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/01/31 23:01:45 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/01/31 23:01:45 INFO mapred.FileInputFormat: Total input paths to process : 1
16/01/31 23:01:45 INFO mapreduce.JobSubmitter: number of splits:1
16/01/31 23:01:46 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/01/31 23:01:46 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/01/31 23:01:46 INFO mapreduce.JobSubmitter: Su

In [16]:
#show results
!hdfs dfs -cat /user/dunmireg/sortedWordCount/part-00000

16/01/31 23:01:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Issue	Count	Relative Count
blank	4	2.96667240223e-06
disclosures	64	4.74667584357e-05
missing	64	4.74667584357e-05
day	71	5.26584351396e-05
amt	71	5.26584351396e-05
checks	75	5.56251075419e-05
convenience	75	5.56251075419e-05
credited	92	6.82334652514e-05
payment	92	6.82334652514e-05
amount	98	7.26834738547e-05
apply	118	8.75168358659e-05
overlimit	127	9.41918487709e-05
stop	131	9.71585211731e-05
charges	131	9.71585211731e-05
sale	139	0.000103091865978
did	139	0.000103091865978
applied	139	0.000103091865978
receive	139	0.000103091865978
acct	163	0.000120891900391
arbitration	168	0.000124600240894
wrong	169	0.000125341908994
bank	202	0.000149816956313
received	216	0.000160200309721
bankruptcy	222	0.000164650318324
advance	240	0.000178000344134
cash	240	0.000178000344134
privacy	240	0.000178000344134
delay	243	0.000180225348436
processing	243

In [24]:
#move output to local directory
#!hadoop fs -copyToLocal /user/dunmireg/word_count

In [17]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/Consumer_Complaints.csv
!hadoop fs -rmr /user/dunmireg/word_count
!hadoop fs -rmr /user/dunmireg/sortedWordCount
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/01/31 23:02:09 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:02:10 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/Consumer_Complaints.csv
rmr: DEPRECATED: Please use 'rm -r' instead.
16/01/31 23:02:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:02:11 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/word_count
rmr: DEPRECATED: Please use 'rm -r' instead.
16/01/31 23:02:12 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/01/31 23:02:13 INFO fs.TrashPolicyDefault: Namenode trash confi

### HW3.3. Shopping Cart Analysis

Product Recommendations: The action or practice of selling additional products or services 
to existing customers is called cross-selling. Giving product recommendation is 
one of the examples of cross-selling that are frequently used by online retailers. 
One simple method to give product recommendations is to recommend products that are frequently
browsed together by the customers.

For this homework use the online browsing behavior dataset located at: 

       https://www.dropbox.com/s/zlfyiwa70poqg74/ProductPurchaseData.txt?dl=0

Each line in this dataset represents a browsing session of a customer. 
On each line, each string of 8 characters represents the id of an item browsed during that session. 
The items are separated by spaces.

Here are the first few lines of the ProductPurchaseData 
FRO11987 ELE17451 ELE89019 SNA90258 GRO99222 
GRO99222 GRO12298 FRO12685 ELE91550 SNA11465 ELE26917 ELE52966 FRO90334 SNA30755 ELE17451 FRO84225 SNA80192 
ELE17451 GRO73461 DAI22896 SNA99873 FRO86643 
ELE17451 ELE37798 FRO86643 GRO56989 ELE23393 SNA11465 
ELE17451 SNA69641 FRO86643 FRO78087 SNA11465 GRO39357 ELE28573 ELE11375 DAI54444 


Do some exploratory data analysis of this dataset. 

How many unique items are available from this supplier?

Using a single reducer: Report your findings such as number of unique products; largest basket; report the top 50 most frequently purchased items,  their frequency,  and their relative frequency (break ties by sorting the products alphabetical order) etc. using Hadoop Map-Reduce. 


In [8]:
%%writefile mapper-3-3.py
#!/usr/bin/python
import sys

total = 0
line_num = 1
for line in sys.stdin:
    line = line.strip()
    items = line.split()
    for item in items:
        total += 1
        print item + '\t' + str(1)
    print '*' + '\t' + str(line_num) + '\t' + str(len(items))
    line_num += 1
print '*' + '\t' + str(0) + '\t' + str(total)

Overwriting mapper-3-3.py


In [15]:
%%writefile reducer-3-3.py
#!/usr/bin/python
import sys
import operator

total = 0
unique_items = set()
largest_basket = {}

current_item = None
current_count = None
item = None

wordcount = {}

for line in sys.stdin:
    line = line.strip()
    line = line.split('\t')
    
    if line[0] == '*':
        if line[1] == '0':
            total = int(line[2])
        else:
            largest_basket[line[1]] = int(line[2])
    else:
        item = line[0]
        count = int(line[1])
        unique_items.add(item)
        
        if current_item == item:
            current_count += count
        else:
            if current_item:
                wordcount[current_item] = current_count
            current_item = item
            current_count = count

if current_item == item:
    wordcount[current_item] = current_count

largest = 50
smallest = 10
#sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sortedWordCount = sorted(wordcount.items(), key = lambda x: (-x[1], x[0]))

print "The Top 50 items are:"
for i in range(largest):
    print sortedWordCount[i][0] + '\t' + str(sortedWordCount[i][1]) + '\t' + str(float(sortedWordCount[i][1])/total)

print '\n'
print "The 10 smalleset items are"
for i in range(smallest):
    print sortedWordCount[-i-1][0] + '\t' + str(sortedWordCount[-i-1][1]) + '\t' + str(float(sortedWordCount[-i-1][1])/total)

print '\n'
print "Number of unique items from this supplier: " + str(len(unique_items))
print "Largest basket is " + max(largest_basket.iteritems(), key = operator.itemgetter(1))[0] + " with a basket size of " + str(max(largest_basket.iteritems(), key = operator.itemgetter(1))[1]) 

#result = max(your_dict.iteritems(), key=operator.itemgetter(1))[0]

Overwriting reducer-3-3.py


In [16]:
!cat ProductPurchaseData.txt | python mapper-3-3.py | sort | python reducer-3-3.py

The Top 50 items are:
DAI62779	6667	0.0175067747831
FRO40251	3881	0.010191059387
ELE17451	3875	0.0101753040775
GRO73461	3602	0.00945843749344
SNA80324	3044	0.00799319370628
ELE32164	2851	0.0074863979161
DAI75645	2736	0.00718442114993
SNA45677	2455	0.0064465474865
FRO31317	2330	0.0061183118711
DAI85309	2293	0.00602115412894
ELE26917	2292	0.00601852824402
FRO80039	2233	0.00586360103355
GRO21487	2115	0.00555374661261
SNA99873	2083	0.00546971829507
GRO59710	2004	0.00526227338613
GRO71621	1920	0.00504169905258
FRO85978	1918	0.00503644728273
GRO30386	1840	0.00483162825872
ELE74009	1816	0.00476860702057
GRO56726	1784	0.00468457870302
DAI63921	1773	0.00465569396887
GRO46854	1756	0.00461105392517
ELE66600	1713	0.00449814087347
DAI83733	1712	0.00449551498855
FRO32293	1702	0.00446925613932
ELE66810	1697	0.0044561267147
SNA55762	1646	0.00432220658362
DAI22177	1627	0.00427231477008
FRO78087	1531	0.00402022981745
ELE99737	1516	0.0039808415436
ELE34057	1489	0.00390994265067
GRO94758	1489	0.0039099426

In [17]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 17:53:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 17:54:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [18]:
#add input to hdfs
!hdfs dfs -put ProductPurchaseData.txt /user/dunmireg

16/02/01 17:54:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-3.py \
-reducer reducer-3-3.py \
-input ProductPurchaseData.txt \
-output sortedWordCount

16/02/01 17:54:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:54:10 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:54:10 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:54:10 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:54:10 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:54:10 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:54:11 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local672224401_0001
16/02/01 17:54:11 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 17:54:11 INFO mapreduce.Job: Running job: job_local672224401_0001
16/02/01 17:54:11 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 17:54:11 

In [20]:
#show results
!hdfs dfs -cat /user/dunmireg/sortedWordCount/part-00000

16/02/01 17:54:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
The Top 50 items are:	
DAI62779	6667	0.0175067747831
FRO40251	3881	0.010191059387
ELE17451	3875	0.0101753040775
GRO73461	3602	0.00945843749344
SNA80324	3044	0.00799319370628
ELE32164	2851	0.0074863979161
DAI75645	2736	0.00718442114993
SNA45677	2455	0.0064465474865
FRO31317	2330	0.0061183118711
DAI85309	2293	0.00602115412894
ELE26917	2292	0.00601852824402
FRO80039	2233	0.00586360103355
GRO21487	2115	0.00555374661261
SNA99873	2083	0.00546971829507
GRO59710	2004	0.00526227338613
GRO71621	1920	0.00504169905258
FRO85978	1918	0.00503644728273
GRO30386	1840	0.00483162825872
ELE74009	1816	0.00476860702057
GRO56726	1784	0.00468457870302
DAI63921	1773	0.00465569396887
GRO46854	1756	0.00461105392517
ELE66600	1713	0.00449814087347
DAI83733	1712	0.00449551498855
FRO32293	1702	0.00446925613932
ELE66810	1697	0.0044561267147
SNA55762	1646	0.00432220658362


In [21]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/ProductPurchaseData.txt
!hadoop fs -rmr /user/dunmireg/sortedWordCount
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:54:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:54:17 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/ProductPurchaseData.txt
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:54:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:54:19 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/sortedWordCount
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/02/01 17:54:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stop

### HW3.4.

(Computationally prohibitive but then again Hadoop can handle this) Pairs

Suppose we want to recommend new products to the customer based on the products they
have already browsed on the online website. Write a map-reduce program 
to find products which are frequently browsed together. Fix the support count (cooccurence count) to s = 100 
(i.e. product pairs need to occur together at least 100 times to be considered frequent) 
and find pairs of items (sometimes referred to itemsets of size 2 in association rule mining) that have a support count of 100 or more.

List the top 50 product pairs with corresponding support count (aka frequency), and relative frequency or support (number of records where they coccur, the number of records where they coccur/the number of baskets in the dataset)  in decreasing order of support  for frequent (100>count) itemsets of size 2. 

Use the Pairs pattern (lecture 3)  to  extract these frequent itemsets of size 2. Free free to use combiners if they bring value. Instrument your code with counters for count the number of times your mapper, combiner and reducers are called.  

Please output records of the following form for the top 50 pairs (itemsets of size 2): 

      item1, item2, support count, support



Fix the ordering of the pairs lexicographically (left to right), 
and break ties in support (between pairs, if any exist) 
by taking the first ones in lexicographically increasing order. 

Report  the compute time for the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)
Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts.

In [22]:
%%writefile mapper-3-4.py
#!/usr/bin/python
import sys
from itertools import combinations

totalBaskets = 0
for line in sys.stdin:
    totalBaskets += 1
    line = line.strip()
    line = line.split()
    
    pairs = list(combinations(line, 2))
    for pair in pairs:
        pair = sorted(list(pair))
        print pair[0] + ' ' + pair[1] + '\t' + str(1)
print '*' + '\t' + str(totalBaskets)

Overwriting mapper-3-4.py


In [23]:
%%writefile reducer-3-4.py
#!/usr/bin/python
import sys
from collections import defaultdict
import operator

support = 100
totalBaskets = 0
pairs = defaultdict(int)

for line in sys.stdin:
    line = line.split('\t')
    if line[0] == '*':
        totalBaskets = int(line[1])
    else:
        pairs[line[0]] += int(line[1])

freqDict = {}
for pair, count in pairs.iteritems():
    if count > 3:
        freqDict[pair] = count

print "Top 50 item pairs:"
print '\n'
print 'Item Pair' + '\t' + 'Support Count' + '\t' + 'Relative Support Count'
print '\n'
        
sortedFreqDict = sorted(freqDict.items(), key = lambda x: (-x[1], x[0]))
for i in range(50):
    print sortedFreqDict[i][0] + '\t' + str(sortedFreqDict[i][1]) + '\t' + str(float(sortedFreqDict[i][1])/totalBaskets)

Overwriting reducer-3-4.py


In [2]:
#!cat ProductPurchaseData.txt | python mapper-3-4.py | sort | python reducer-3-4.py > output.txt

In [24]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 17:56:06 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 17:56:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [25]:
#add input to hdfs
!hdfs dfs -put ProductPurchaseData.txt /user/dunmireg

16/02/01 17:56:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
time !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-4.py \
-reducer reducer-3-4.py \
-input ProductPurchaseData.txt \
-output sortedProducts

16/02/01 17:56:36 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:56:36 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 17:56:36 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 17:56:36 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 17:56:37 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 17:56:37 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 17:56:37 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local350127535_0001
16/02/01 17:56:37 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 17:56:37 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.FileOutputCommitter
16/02/01 17:56:37 INFO mapreduce.Job: The url to track the job: http:/

In [27]:
#show results
!hdfs dfs -cat /user/dunmireg/sortedProducts/part-00000

16/02/01 17:57:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Top 50 item pairs:	
	
	
Item Pair	Support Count	Relative Support Count
	
	
DAI62779 ELE17451	1592	0.0511880646925
FRO40251 SNA80324	1412	0.0454004694383
DAI75645 FRO40251	1254	0.0403202469374
FRO40251 GRO85051	1213	0.0390019613517
DAI62779 GRO73461	1139	0.0366226166361
DAI75645 SNA80324	1130	0.0363332368734
DAI62779 FRO40251	1070	0.0344040384554
DAI62779 SNA80324	923	0.0296775023311
DAI62779 DAI85309	918	0.0295167357963
ELE32164 GRO59710	911	0.0292916626475
DAI62779 DAI75645	882	0.0283592167454
FRO40251 GRO73461	882	0.0283592167454
DAI62779 ELE92920	877	0.0281984502106
FRO40251 FRO92469	835	0.026848011318
DAI62779 ELE32164	832	0.0267515513971
DAI75645 GRO73461	712	0.0228931545609
DAI43223 ELE32164	711	0.022861001254
DAI62779 GRO30386	709	0.02279669464
ELE17451 FRO40251	697	0.0224108549564
DAI85309 ELE99737	659	0.0211890292917
DAI62779 ELE26

In [28]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/ProductPurchaseData.txt
!hadoop fs -rmr /user/dunmireg/sortedProducts
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:57:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:57:12 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/ProductPurchaseData.txt
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 17:57:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 17:57:13 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/sortedProducts
stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
16/02/01 17:57:25 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopp

### HW3.5: Stripes
Repeat 3.4 using the stripes design pattern for finding cooccuring pairs.

Report  the compute times for stripes job versus the Pairs job. Describe the computational setup used (E.g., single computer; dual core; linux, number of mappers, number of reducers)

Instrument your mapper, combiner, and reducer to count how many times each is called using Counters and report these counts. Discuss the differences in these counts between the Pairs and Stripes jobs


In [21]:
%%writefile mapper-3-5.py
#!/usr/bin/python
import sys

totalBaskets = 0
for line in sys.stdin:
    line = line.strip()
    line = line.split()
    totalBaskets += 1
    
    for item in line:
        counts = {}
        for othItem in line:
            if othItem != item:
                if othItem in counts.keys():
                    counts[othItem] += 1
                else:
                    counts[othItem] = 1
        print item + '\t' + str(counts)
print "*" + '\t' + str(totalBaskets)

Overwriting mapper-3-5.py


In [22]:
%%writefile reducer-3-5.py
#!/usr/bin/python
import sys
import ast
from collections import Counter, OrderedDict

support = 3
totalBaskets = 0
current_item = None
current_item_dict = Counter({})

for line in sys.stdin:
    line = line.strip().split('\t')
    
    if line[0] == '*':
        totalBaskets = int(line[1])
    else:
        item = line[0]
        item_dict = line[1]
        item_dict = Counter(eval(item_dict))
        if current_item == item:
            current_item_dict += Counter(item_dict)
        else:
            if current_item:
                for i in OrderedDict(sorted(current_item_dict.items())):
                    if current_item_dict[i] >= support:
                        print current_item + ' ' + i + '\t' + str(current_item_dict[i]) + '\t' + str(float(current_item_dict[i])/totalBaskets)
            current_item = item
            current_item_dict = item_dict
            
for i in OrderedDict(sorted(current_item_dict.items())):
    if current_item_dict[i] >= support:
        print current_item + ' ' + i + '\t' + str(current_item_dict[i]) + '\t' + str(float(current_item_dict[i])/totalBaskets)

Overwriting reducer-3-5.py


In [23]:
%%writefile mapper-3-5-2.py
#!/usr/bin/python
import sys

#Structure is word + \t + count + \t + relative count

for line in sys.stdin:
    line = line.split('\t')
    print line[1] + '\t'+ line[0] + '\t' + line[2]

Overwriting mapper-3-5-2.py


In [24]:
%%writefile reducer-3-5-2.py
#!/usr/bin/python
import sys

print "Pair" + '\t' + "Count" + '\t' + 'Relative Count'
for line in sys.stdin:
    line = line.split('\t')
    print line
    #structure currently is count + word + relative count
    print line[1] + '\t' + line[0] + '\t'+ line[2]

Overwriting reducer-3-5-2.py


__Problem where sort -n is outputting a blank file. Unclear why__

In [26]:
!cat ProductPurchaseData2.txt | python mapper-3-5.py | sort | python reducer-3-5.py | python mapper-3-5-2.py > output.txt #python reducer-3-5-2.py > output.txt

In [25]:
#Start hadoop yarn
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-resourcemanager-Glenns-Air.home.out
localhost: starting nodemanager, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/yarn-dunmireg-nodemanager-Glenns-Air.home.out
16/02/01 22:11:45 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-namenode-Glenns-Air.home.out
localhost: starting datanode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-datanode-Glenns-Air.home.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/Cellar/hadoop/2.7.1/libexec/logs/hadoop-dunmireg-secondarynamenode-Glenns-Air.home.out
16/02/01 22:12:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for

In [26]:
#add input to hdfs
!hdfs dfs -put ProductPurchaseData.txt /user/dunmireg

16/02/01 22:12:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
time !hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-mapper mapper-3-5.py \
-reducer reducer-3-5.py \
-input ProductPurchaseData.txt \
-output sortedProducts

16/02/01 22:12:31 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:12:31 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:12:31 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:12:31 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:12:32 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:12:32 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:12:32 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local500015672_0001
16/02/01 22:12:32 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
16/02/01 22:12:32 INFO mapred.LocalJobRunner: OutputCommitter set in config null
16/02/01 22:12:32 INFO mapred.LocalJobRunner: OutputCommitter is org.apache.hadoop.mapred.Fil

In [31]:
#run second job to properly sort by counts
!hadoop jar /usr/local/Cellar/hadoop/2.7.1/libexec/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D  mapred.text.key.comparator.options=-n \
-mapper mapper-3-5-2.py \
-reducer reducer-3-5-2.py \
-input /user/dunmireg/sortedProducts/part-00000 \
-output numSortedProducts

16/02/01 22:20:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:20:56 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
16/02/01 22:20:56 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
16/02/01 22:20:56 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
16/02/01 22:20:56 INFO mapred.FileInputFormat: Total input paths to process : 1
16/02/01 22:20:56 INFO mapreduce.JobSubmitter: number of splits:1
16/02/01 22:20:56 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
16/02/01 22:20:56 INFO Configuration.deprecation: mapred.output.key.comparator.class is deprecated. Instead, use mapreduce.job.output.key.comparator.class
16/02/01 22:20:56 INFO mapreduce.JobSubmitter: Su

In [46]:
#Remove output directory and stop yarn and hdfs
!hadoop fs -rmr /user/dunmireg/ProductPurchaseData.txt
!hadoop fs -rmr /user/dunmireg/sortedProducts
!hadoop fs -rmr /user/dunmireg/numSortedProducts
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-yarn.sh
!/usr/local/Cellar/hadoop/2.7.1/sbin/stop-dfs.sh

rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:27:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:27:23 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/ProductPurchaseData.txt
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:27:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:27:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/dunmireg/sortedProducts
rmr: DEPRECATED: Please use 'rm -r' instead.
16/02/01 22:27:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/02/01 22:27:27 INFO fs.TrashPolicyDefault: Namenode trash c